In [30]:
import asyncio
import httpx
from bs4 import BeautifulSoup
from tqdm.asyncio import tqdm_asyncio
from pathlib import Path
import time
import requests

In [31]:
import requests
from bs4 import BeautifulSoup


def get_search_url(keyword):
    session = requests.Session()

    # 1. Visit search page to get cookies + CSRF token
    search_page_url = "https://www.blacktowhite.net/search/?type=xfmg_media"
    resp = session.get(
        search_page_url,
        headers={
            "User-Agent": "Mozilla/5.0",
        },
    )

    # Parse CSRF token from HTML
    soup = BeautifulSoup(resp.text, "html.parser")
    token_input = soup.find("input", {"name": "_xfToken"})

    if not token_input:
        raise Exception(
            "Could not locate CSRF token. Site may require age verification."
        )

    xf_token = token_input.get("value")
    print("CSRF token:", xf_token)

    # 2. Now prepare the POST data for the search
    data = [
        ("_xfToken", xf_token),
        ("keywords", keyword),
        ("c[users]", ""),
        ("c[newer_than]", ""),
        ("c[older_than]", ""),
        ("c[categories][]", "5"),
        ("c[child_categories]", "1"),
        ("order", "relevance"),
        ("search_type", "xfmg_media"),
        ("_xfRequestUri", "/search/?type=xfmg_media"),
        ("_xfWithData", "1"),
        ("_xfToken", xf_token),
        ("_xfResponseType", "json"),
    ]

    # 3. Send the search request (with cookies + token)
    headers = {
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "User-Agent": "Mozilla/5.0",
        "X-Requested-With": "XMLHttpRequest",
        "Origin": "https://www.blacktowhite.net",
        "Referer": search_page_url,
    }

    response = session.post(
        "https://www.blacktowhite.net/search/search", headers=headers, data=data
    )

    response.raise_for_status()
    return response.json().get("redirect")


In [32]:
get_search_url("pawg")

KeyboardInterrupt: 

In [42]:
BASE_URL = "https://www.blacktowhite.net"
HEADERS = {"User-Agent": "Mozilla/5.0"}


async def fetch(client, url):
    response = await client.get(url, follow_redirects=True)
    return response.text


async def fetch_video_src(client, video_page_url):
    response = await client.get(video_page_url)
    soup = BeautifulSoup(response.text, "html.parser")
    source = soup.find("source")
    if source and source.get("src"):
        return source["src"]
    return None


async def get_video_links_from_page(client, page, category, search_url=""):
    url = (
        f"{BASE_URL}/{category}/page-{page}?order=view_count&direction=desc&type=video&newer_than=all"
        if not search_url
        else search_url + f"&page={page}"
    )
    html = await fetch(client, url)
    soup = BeautifulSoup(html, "html.parser")
    if search_url:
        tasks = [
            f"{BASE_URL}{i.find('a')['href']}"
            for i in soup.find_all("h3")
            if i.find("a") and i.find("a").has_attr("href")
        ]
        tasks = [
            fetch_video_src(client, f"{BASE_URL}{i.find('a')['href']}")
            for i in soup.find_all("h3")
            if i.find("a") and i.find("a").has_attr("href")
        ]
    else:
        tasks = []
        for a_tag in soup.find_all("a"):
            if a_tag.has_attr("data-src"):
                video_page_url = BASE_URL + a_tag["data-src"]
                tasks.append(fetch_video_src(client, video_page_url))

    return await asyncio.gather(*tasks)


async def main(category, search=False, search_url=""):
    videos = []
    async with httpx.AsyncClient(headers=HEADERS, timeout=30) as client:
        if search:
            search_url = get_search_url(category)
            print(f"{search_url = }")
        tasks = [
            get_video_links_from_page(client, page, category, search_url)
            for page in range(1, 11)
        ]
        results = await asyncio.gather(*tasks)
        for result in results:
            videos.extend(filter(None, result))  # Filter out any Nones

    print(f"Total videos found: {len(videos)}")
    return videos


categories, search = (
    ["cuckold-vodeos", "blowjob-videos", "fucking-videos", "videos"],
    False,
)
categories, search = (
    ["teen", "school"],
    True,
)
for category in categories:
    print(category)
    time.sleep(5)
    videos = await main(category, search)
    html = Path("template.html").read_text()
    with open(f"{category}.html", "w", encoding="utf-8") as f:
        f.write(
            html.replace(
                "<--videos-->",
                " ".join(
                    [
                        f"""
    <div class="video-tile" data-video-url="{BASE_URL}{url}">
      <a href="{BASE_URL}{url}" target="_blank">
        <video src="{BASE_URL}{url}" preload="metadata" muted></video>
        <span class="duration">Loading...</span>
      </a>
    </div>"""
                        for url in videos
                    ]
                ),
            )
        )


teen
CSRF token: 1763458399,30f7f1fc77b32cb52446a700586a78cf
search_url = 'https://www.blacktowhite.net/search/59304956/?q=teen&t=xfmg_media&c[categories][0]=5&c[child_categories]=1&o=relevance'
Total videos found: 200
school
CSRF token: 1763458426,0b242d50f20be0d33629c9b21d9e5924
search_url = 'https://www.blacktowhite.net/search/59305137/?q=school&t=xfmg_media&c[categories][0]=5&c[child_categories]=1&o=relevance'
Total videos found: 134
